In [1]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
import pandas as pd
import numpy as np
# train 변수 : 문제 가져오기
train = pd.read_csv('data/kaggle/train.csv')
# test 변수 : 정답 가져오기
test = pd.read_csv('data/kaggle/test.csv')


In [17]:
# 필요없는 컬럼 drop
train = train.drop(['no','fnlwgt','capital-gain', 'capital-loss', 'education-num', 'income'],axis=1)
test = test.drop(['no','fnlwgt','capital-gain', 'capital-loss', 'education-num'],axis=1)


In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29305 entries, 0 to 29304
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             29305 non-null  int64 
 1   workclass       29305 non-null  object
 2   education       29305 non-null  object
 3   marital-status  29305 non-null  object
 4   occupation      29305 non-null  object
 5   relationship    29305 non-null  object
 6   race            29305 non-null  object
 7   sex             29305 non-null  object
 8   hours-per-week  29305 non-null  int64 
 9   native-country  29305 non-null  object
dtypes: int64(2), object(8)
memory usage: 2.2+ MB


In [4]:
# 전처리 되기 이전의 feature들 (전처리 되기전 문제들)
features_raw = data_train.drop('income', axis = 1)

skewed = ['capital-gain', 'capital-loss']
features_log_transformed = pd.DataFrame(data = features_raw)
features_log_transformed[skewed] = features_raw[skewed].apply(lambda x: np.log(x + 1))

In [5]:
# min max 스케일러를 이용하여 특징 벡터 (문제들) 스케일링 (최소 최대 정규화)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

features_log_minmax_transform = pd.DataFrame(data = features_log_transformed)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_transformed[numerical])

In [6]:

# 자산이득액 에서 자산손해액을 뺴서 순이익을 구하기
data_train['total_cap'] = data_train['capital-gain'] - data_train['capital-loss']
data_test['total_cap'] = data_test['capital-gain'] - data_test['capital-loss']

#  순이익 계산후 필요없어진 컬럼 삭제
data_train = data_train.drop(['capital-gain','capital-loss'],axis=1)
data_test = data_test.drop(['capital-gain','capital-loss'],axis=1)

In [7]:
# 정답 데이터 분리
y_train = data_train['income']
data_train.drop('income',axis=1,inplace=True)
#원 핫 인코딩  == object타입 정수형데이터로 변경
OH = ['workclass','marital-status','relationship','race','sex']

X_train = pd.get_dummies(data_train)
X_test = pd.get_dummies(data_test)

In [8]:
# lgb 모델 사용하여 예측
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=200, n_jobs=-1, reg_alpha=0.9, reg_lambda=0.2, n_estimators=200)

# lgb 모델 훈련
lgb.fit(X_train, y_train)

# 훈련한 lgb 모델로 예측한 결과
y_pred = lgb.predict(X_test)

rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = y_pred
rs.to_csv('50K_01.csv', index=False)


In [9]:
# 방식은 이거임 -> 테스트 데이터도 훈련 데이터로 이용하자 테스트 데이터에 존재 하지 않는 정답 데이터는 처음에 훈련 데이터로 예측한 테스트 데이터 값을 정답 데이터로 사용하자
# 훈련 데이터와, 테스트 데이터 합치기 (테스트 데이터 정답은 훈련된 모델의 예측한 결과를 사용)
X_train_2 = pd.concat([X_train,X_test])
X_train_2

,age,education-num,hours-per-week,total_cap,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,...,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male
0,25,7,40,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
1,39,10,55,-1721,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
2,35,13,50,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
3,64,11,40,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
4,24,10,70,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19532,45,9,60,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
19533,48,9,50,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
19534,63,10,40,4386,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
19535,18,7,20,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,1,0


In [10]:
# 방식은 이거임 -> 테스트 데이터도 훈련 데이터로 이용하자 테스트 데이터에 존재 하지 않는 정답 데이터는 처음에 훈련 데이터로 예측한 테스트 데이터 값을 정답 데이터로 사용하자
# 훈련 데이터와, 테스트 데이터 합치기 (테스트 데이터 정답은 훈련된 모델의 예측한 결과를 사용)
X_train_2 = pd.concat([X_train,X_test])

In [11]:
y_train

0        0
1        0
2        0
3        0
4        1
        ..
29300    0
29301    0
29302    0
29303    0
29304    0
Name: income, Length: 29305, dtype: int64

In [12]:
y_1=y_train.to_numpy()

In [13]:
y_t = np.concatenate([y_1, y_pred])

In [14]:
y_t = list(y_t)

In [15]:
y_train_change=pd.Series(y_t)

In [16]:
### 모델 훈련
lgb.fit(X_train_2,y_train_change)

LGBMClassifier(n_estimators=200, random_state=200, reg_alpha=0.9,
               reg_lambda=0.2)

In [17]:
# lgb 모델로 예측한 결과
pre1 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre1 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Test01.csv', index = False)

In [18]:
# 위 과정 반복 -> 모델 예측한 결과중 더 좋은 결과를 다시 테스트 데이터의 정답으로 활용 + 트레인 데이터와 다시 합치고 다시 모델 학습
y_t = np.concatenate([y_1, pre1])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre2 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre2 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Test02.csv', index = False)

In [19]:
y_t = np.concatenate([y_1, pre2])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre3 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre3 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last01.csv', index = False)

In [20]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88013103 0.88238305 0.87940213 0.97645373 0.99160524]
평균 정확도: 0.9219950345193461


In [21]:
y_t = np.concatenate([y_1, pre3])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre4 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre4 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Test04.csv', index = False)


In [22]:

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88033576 0.87982393 0.88011876 0.97727273 0.99180999]
평균 정확도: 0.9218722326026043


In [23]:
y_t = np.concatenate([y_1, pre4])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre5 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre5 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Test05.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.87982393 0.88054049 0.87970925 0.97757985 0.99283374]
평균 정확도: 0.9220974536359939


In [24]:
y_t = np.concatenate([y_1, pre5])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre6 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre6 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last06.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.8796192  0.88074521 0.87940213 0.9781941  0.99232187]
평균 정확도: 0.922056503595044


In [25]:
y_t = np.concatenate([y_1, pre6])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre7 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre7 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last07.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.87920975 0.88033576 0.88032351 0.97809173 0.99150287]
평균 정확도: 0.9218927201985856


In [26]:
y_t = np.concatenate([y_1, pre7])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre8 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre8 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last08.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88002866 0.88084758 0.88011876 0.97850123 0.99150287]
평균 정확도: 0.9221998182587802


In [27]:
y_t = np.concatenate([y_1, pre8])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre9 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre9 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last09.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.8796192  0.88043812 0.879095   0.97952498 0.99201474]
평균 정확도: 0.9221384099646972


In [28]:
y_t = np.concatenate([y_1, pre9])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre10 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre10 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last10.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88002866 0.88084758 0.879914   0.98013923 0.99037674]
평균 정확도: 0.9222612433202053


In [29]:
y_t = np.concatenate([y_1, pre10])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre11 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre11 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last11.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.87951684 0.88094994 0.879095   0.97880835 0.99242424]
평균 정확도: 0.9221588766015012


In [30]:
y_t = np.concatenate([y_1, pre11])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre12 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre12 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last12.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88023339 0.88064285 0.87970925 0.97921785 0.99088862]
평균 정확도: 0.9221383931973552


In [31]:
y_t = np.concatenate([y_1, pre12])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre13 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre13 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last13.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88094994 0.87931211 0.88032351 0.97880835 0.99283374]
평균 정확도: 0.9224455310799871


In [32]:
y_t = np.concatenate([y_1, pre13])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre14 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre14 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last14.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.8799263  0.87890265 0.87940213 0.97921785 0.99221949]
평균 정확도: 0.9219336849109598


In [33]:
y_t = np.concatenate([y_1, pre14])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre15 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre15 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last15.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88033576 0.88054049 0.87940213 0.97788698 0.99211712]
평균 정확도: 0.922056493115455


In [34]:
y_t = np.concatenate([y_1, pre15])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre16 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre16 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last16.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88074521 0.87890265 0.87929975 0.97911548 0.99119574]
평균 정확도: 0.9218517680617179


In [35]:
y_t = np.concatenate([y_1, pre16])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre17 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre17 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last17.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.87982393 0.8799263  0.87929975 0.9794226  0.99283374]
평균 정확도: 0.9222612663753006


In [36]:
y_t = np.concatenate([y_1, pre17])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre18 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre18 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last18.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.88094994 0.88013103 0.87940213 0.97911548 0.99201474]
평균 정확도: 0.9223226641897948


In [37]:
y_t = np.concatenate([y_1, pre18])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre19 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre19 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last19.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.87982393 0.87972157 0.87960688 0.98024161 0.99140049]
평균 정확도: 0.9221588954647609


In [38]:
y_t = np.concatenate([y_1, pre19])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre20 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre20 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last20.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

교차 검증 정확도: [0.87982393 0.8799263  0.87899263 0.98085586 0.99191237]
평균 정확도: 0.9223022164162507


In [39]:
y_t = np.concatenate([y_1, pre20])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre21 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre21 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last21.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

KeyboardInterrupt: 

In [ ]:
y_t = np.concatenate([y_1, pre21])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre22 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre22 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last22.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

In [ ]:
y_t = np.concatenate([y_1, pre22])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre23 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre23 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last23.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

In [ ]:
y_t = np.concatenate([y_1, pre23])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre24 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre24 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last24.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

In [ ]:
y_t = np.concatenate([y_1, pre24])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre25 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre25 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last25.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())

In [ ]:
y_t = np.concatenate([y_1, pre25])
y_t = list(y_t)
y_train_change=pd.Series(y_t)

lgb.fit(X_train_2,y_train_change)

pre26 = lgb.predict(X_test)
# kaggle 양식 불러오기
rs =  pd.read_csv('./data/kaggle/sample_submission.csv')
rs['income'] = pre26 # 내 모델의 예측 결과를 양식에 넣기
rs.to_csv('Last26.csv', index = False)

scores = cross_val_score(lgb, X_train_2, y_train_change, cv=5, scoring='accuracy')

# 교차 검증 결과 출력
print("교차 검증 정확도:", scores)
print("평균 정확도:", scores.mean())